## EDA on ACLU Bill Data

In [46]:
# IMPORT PACKAGES
import pandas as pd
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import re
import wordninja

In [29]:
# LOAD IN THE DATA
data = pd.read_csv("../modified_data/merged_bill_data.csv")
# Drop the unnamed column
data = data.drop(data.columns[0], axis = 1)
# Get the number of characters in each bill
data["number_characters"] = data["text"].str.len()

### Tokenize Option 1: Tokenizing with NLTK

In [33]:
data["nltk_tokens"] = [word_tokenize(text) for text in data["text"]]
# Get the number of tokens with NLTK in each bill
data["number_nltk_tokens"] = data["nltk_tokens"].str.len()
# Get the number of bills with less than or equal to 512 tokens
len(data[(data["number_nltk_tokens"] < 512)])

### Tokenize Option 2: Removing Whitespace + Word Ninja Inference Tokenization

In [49]:
# Make a column of the text without spaces
data["text_no_spaces"] = data["text"].str.replace(r'\s+', '')
# Make a new column of infered words
data["infered_wordninja_words"] = ""
# Infer the words from this new column
for i in range(len(data.text_no_spaces)):
    data.infered_wordninja_words[i] = wordninja.split(data.text_no_spaces[i])

/var/folders/_p/d81k_b_93575z7h2220jh4cr0000gn/T/ipykernel_6376/1212223077.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data["text_no_spaces"] = data["text"].str.replace(r'\s+', '')
/var/folders/_p/d81k_b_93575z7h2220jh4cr0000gn/T/ipykernel_6376/1212223077.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.infered_wordninja_words[i] = wordninja.split(data.text_no_spaces[i])


In [52]:
# Get the number of tokens with wordninja method
data["number_wordninja_tokens"] = data["infered_wordninja_words"].str.len()
# Get the number of bills with less than or equal to 512 tokens
len(data[(data["number_wordninja_tokens"] < 512)])

85